<a href="https://www.kaggle.com/code/nadirfartas/pfenotebook?scriptVersionId=236321355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

secret_label = "HF Hub"
secret_value = UserSecretsClient().get_secret(secret_label)
login(token=secret_value)

In [3]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
!pip install \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    evaluate \
    accelerate \
    bitsandbytes \
    trl \
    peft \
    -U --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 94.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 85.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 52.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 92.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 112.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 107.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 39.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.4 MB/s eta 0:00:00:00:01
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build int

In [5]:
import transformers
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [6]:
def format_squad_dataset(dataset_split):
    """
    Format a SQuAD-style dataset split into a list of dictionaries 
    with context, questions, and answers.

    Args:
        dataset_split: a split of the SQuAD-style dataset, e.g., ds["train"] or ds["validation"]

    Returns:
        A list of formatted entries: [{"context": ..., "questions": [...], "answers": [...]}]
    """
    formatted = []

    for article in dataset_split:
        for paragraph in article["data"]:
            for p in paragraph["paragraphs"]:
                context = p["context"]
                questions = []
                answers = []

                for qa in p["qas"]:
                    if qa["is_impossible"]:
                        continue

                    if not qa.get("answers") or not qa["answers"][0].get("text"):
                        continue

                    question = qa["question"]
                    answer = qa["answers"][0]["text"]

                    questions.append(question)
                    answers.append(answer)

                if questions and answers:
                    formatted.append({
                        "context": context,
                        "questions": questions,
                        "answers": answers
                    })

    return formatted

In [7]:
from datasets import load_dataset
import json

# Load dataset
ds = load_dataset("ZeyadAhmed/Arabic-SQuADv2.0")
formatted_train_dataset = format_squad_dataset(ds["train"])
formatted_val_dataset = format_squad_dataset(ds["validation"])

print(f"✅ Done. Formatted {len(formatted_train_dataset)} training context blocks.")
print(f"✅ Done. Formatted {len(formatted_val_dataset)} validation context blocks.")

asquadv2-train.json:   0%|          | 0.00/91.9M [00:00<?, ?B/s]

asquadv2-val.json:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

asquadv2-test.json:   0%|          | 0.00/27.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

✅ Done. Formatted 17180 training context blocks.
✅ Done. Formatted 4639 validation context blocks.


In [8]:
def flatten_context_qa_pairs(formatted_data):
    """
    Convert formatted context-question-answer data into a flat list of
    input-target training pairs for seq2seq modeling.

    Args:
        formatted_data: List of dicts with 'context', 'questions', and 'answers'

    Returns:
        A list of dicts with 'input' and 'target' fields
    """
    flat_data = []

    for item in formatted_data:
        context = item["context"]
        for question, answer in zip(item["questions"], item["answers"]):
            input_text = f"context: {context} answer: {answer}"
            target_text = question
            flat_data.append({"input": input_text, "target": target_text})

    return flat_data

In [9]:
train_dataset = flatten_context_qa_pairs(formatted_train_dataset)
val_dataset = flatten_context_qa_pairs(formatted_val_dataset)
print(f"Total training pairs: {len(train_dataset)}")
print(f"Total validation pairs: {len(val_dataset)}")
print(train_dataset[1])
print(val_dataset[1])

Total training pairs: 42033
Total validation pairs: 5254
{'input': 'context: ( لم يكن زلزال Ms 6 . 1 في 30 أغسطس 2008 في جنوب سيتشوان جزءا من هذه السلسلة لأنه كان ناجما عن صدع مختلف . انظر زلزال بانتشيهوا 2008 للحصول على التفاصيل . ) answer: زلزال بانتشيهوا 2008', 'target': 'أين يجب أن تبحث عن مزيد من التفاصيل ؟'}
{'input': 'context: غادر فريق الإغاثة الطارئة من الزلازل المكون من 184 شخصا ( يتألف من 12 شخصا من مكتب الدولة لرصد الزلازل و 150 من قيادة منطقة بكين العسكرية و 22 من مستشفى الشرطة العامة المسلحة ) بكين من مطار نانيوان في أواخر 12 مايو في طائرتي نقل عسكريتين للسفر إلى مقاطعة ونتشوان . answer: 184', 'target': 'كم عدد الأشخاص الذين شكلوا فريق الإغاثة ؟'}


In [14]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("UBC-NLP/AraT5v2-base-1024", legacy=False)
model = AutoModelForSeq2SeqLM.from_pretrained("UBC-NLP/AraT5v2-base-1024")
# Convert list to Hugging Face Dataset
hf_train_dataset = Dataset.from_list(train_dataset)
hf_val_dataset = Dataset.from_list(val_dataset)

# Tokenize function
def tokenize(example):
    model_input = tokenizer(
        example["input"], 
        max_length=512, 
        padding="max_length", 
        truncation=True
    )
    labels = tokenizer(
        example["target"], 
        max_length=64, 
        padding="max_length", 
        truncation=True
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

# Apply tokenizer
tokenized_train_dataset = hf_train_dataset.map(tokenize, batched=False)
tokenized_val_dataset = hf_val_dataset.map(tokenize, batched=False)

Map:   0%|          | 0/42033 [00:00<?, ? examples/s]

Map:   0%|          | 0/5254 [00:00<?, ? examples/s]

In [15]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

CUDA available: True
Device: cuda


In [16]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./araT5-qg",
    run_name="araT5-qg-run-1",
    learning_rate=2e-5,                           # standard LR for T5 fine-tuning
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,

    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=1,                           # keep only 2 latest checkpoints
    eval_strategy="epoch",           # use eval_strategy (not evaluation_strategy)
    save_strategy="epoch",
    fp16=True, 
    metric_for_best_model="loss",
    greater_is_better=False,
    load_best_model_at_end=False,
    report_to="none",                             # no wandb/huggingface reporting
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save final model and tokenizer
model.save_pretrained("./araT5-qg-final")
tokenizer.save_pretrained("./araT5-qg-final")

print("✅ Training complete! Model and tokenizer saved to './araT5-qg-final'")

/tmp/ipykernel_73/540974889.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.486000,0.391099
2,0.453800,0.377061
3,0.445300,0.373185


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Training complete! Model and tokenizer saved to 'https://github.com/nadirfartas/araT5-qg-final'


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))